In [8]:
import tkinter
import numpy as np
from datetime import datetime
from tkinter import ttk
from tkinter import scrolledtext as tkst
from tkinter import messagebox as mb
from tkinter.filedialog import askopenfilename
from matplotlib.backend_bases import key_press_handler
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure
import serial.tools.list_ports
import threading
from tkinter import *
import serial
import datetime as dt
import csv
import time

In [9]:
font9 = "-family {Showcard Gothic} -size 14 -weight bold " \
        "-slant roman -underline 0 -overstrike 0"

# ------------------------------------------------Make a Tkinter Canvas---------------------------------------------------
windows = tkinter.Tk()
windows.title("GUI Adaptif Kontrol Eksoskeleton")
windows.configure(background="#e2e6e2")
windows.geometry("+250+10")

# ---------------------------------------------------------Declare Attribute--------------------------------------------------------
Title = tkinter.Label(windows, text="GUI Eksoskeleton", font=font9,
                      bg='#e2e6e2')
Title.pack(pady=10, expand=tkinter.YES, side=tkinter.TOP)
part_1 = tkinter.Frame(windows, background='#e2e6e2')
part_1.pack()

#------------------------------------------------------connect to exoskeleton------------------------------------------------------
my_frame_1 = tkinter.LabelFrame(part_1, width=250, height=100, text='''Connect to Exoskeleton''', relief='groove', foreground="black",
                                background='#e2e6e2')
my_frame_1.pack(padx=5, side=tkinter.LEFT)
my_frame_1.pack_propagate(0)

def connect_check(args):
    if "-" in clicked_bd.get():
        connect_btn["state"] = "disable"
    else:
        connect_btn["state"] = "active"

def baud_select():
    global clicked_bd, drop_bd
    clicked_bd = StringVar()
    bds = ["-",
           "9600",
           "56000",
           "115200"]
    clicked_bd.set(bds[0])
    drop_bd = OptionMenu(my_frame_1, clicked_bd, *bds, command=connect_check)
    drop_bd.config(width=7)
    drop_bd.grid(column=2, row=3, padx=10)

def update_coms():
    global clicked_com, drop_COM
    ports = serial.tools.list_ports.comports()
    coms = [com[0] for com in ports]
    coms.insert(0, "-")
    try:
        drop_COM.destroy()
    except:
        pass
    clicked_com = StringVar()
    clicked_com.set(coms[0])
    drop_COM = OptionMenu(my_frame_1, clicked_com, *coms, command=connect_check)
    drop_COM.config(width=7)
    drop_COM.grid(column=2, row=2, padx=10)
    connect_check(0)

def connexion():
    global ser, serialData
    if connect_btn["text"] in "Disconnect":
        serialData = False
        connect_btn["text"] = "Connect"
        refresh_btn["state"] = "active"
        drop_bd["state"] = "active"
        drop_COM["state"] = "active"

    else:
        serialData = True
        connect_btn["text"] = "Disconnect"
        refresh_btn["state"] = "disable"
        drop_bd["state"] = "disable"
        drop_COM["state"] = "disable"
        port = clicked_com.get()
        baud = clicked_bd.get()
        try:
            ser = serial.Serial(port, baud, timeout=.1)
        except:
            pass
        t1 = threading.Thread(target=readSerial)
        t1.deamon = True
        t1.start()      

def readSerial():
    global serialData, kec, arus
    while serialData:
        data = ser.readline()
        f = open("GUI1.csv", "a+")
        writer = csv.writer(f, delimiter=',')
        if len(data)>0:
            try:
                sensor = data.decode('utf8')
                string = sensor.rstrip() # remove \n and \r
                if string != "":
                    writer = csv.writer(f, delimiter=',')
                    readOut = ser.readline().decode('ascii')
                    string = readOut.rstrip() # remove \n and \r
                    rows = [float(x) for x in string.split(',')]
                    row = string.split(',')
                    waktu = dt.datetime.now().strftime('%M:%S.%f')
                    rows.insert(0, waktu)
                    writer.writerow(rows)
                    f.flush()
                print(string) 
                my_frame_4_spinbox1.delete(0, tkinter.END)
                my_frame_4_spinbox1.insert(0, int(row[0]))
                my_frame_4_spinbox2.delete(0, tkinter.END)
                my_frame_4_spinbox2.insert(0, int(row[1]))
                my_frame_4_spinbox5.delete(0, tkinter.END)
                my_frame_4_spinbox5.insert(0, int(row[2]))
                my_frame_4_spinbox6.delete(0, tkinter.END)
                my_frame_4_spinbox6.insert(0, int(row[3]))
                my_frame_4_spinbox7.delete(0, tkinter.END)
                my_frame_4_spinbox7.insert(0, int(row[4]))

                kec = (my_frame_4_spinbox1.get())
                arus = (my_frame_4_spinbox2.get())

            except:
                pass

#====================================================================read data motor 1=====================
part_2 = tkinter.Frame(windows, background='#e2e6e2')
part_2.pack()

my_frame_4 = tkinter.Frame(part_2, width=400, height=625, background='#e2e6e2')
my_frame_4 = tkinter.LabelFrame(part_2, width=200, height=200, text='''Data Reading : Motor 1''', relief='groove', foreground="black",
                                background='#e2e6e2')
my_frame_4.pack(padx=5, pady=5, side=tkinter.LEFT)
my_frame_4.pack_propagate(0)

my_frame_4_bantuan1 = tkinter.Frame(my_frame_4, width=85, height=50, background='#e2e6e2')
my_frame_4_bantuan1.pack(side=tkinter.LEFT)
my_frame_4_bantuan2 = tkinter.Frame(my_frame_4, width=65, height=50, background='#e2e6e2')
my_frame_4_bantuan2.pack(side=tkinter.LEFT)
my_frame_4_bantuan9 = tkinter.Frame(my_frame_4_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_4_bantuan9.pack()
my_frame_4_bantuan9.pack_propagate(0)
my_frame_4_bantuan10 = tkinter.Frame(my_frame_4_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_4_bantuan10.pack()
my_frame_4_bantuan10.pack_propagate(0)

my_frame_4_bantuan7 = tkinter.Frame(my_frame_4_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_4_bantuan7.pack()
my_frame_4_bantuan7.pack_propagate(0)
my_frame_4_bantuan8 = tkinter.Frame(my_frame_4_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_4_bantuan8.pack()
my_frame_4_bantuan8.pack_propagate(0)
my_frame_4_bantuan19 = tkinter.Frame(my_frame_4_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_4_bantuan19.pack()
my_frame_4_bantuan19.pack_propagate(0)
my_frame_4_bantuan21 = tkinter.Frame(my_frame_4_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_4_bantuan21.pack()
my_frame_4_bantuan21.pack_propagate(0)
my_frame_4_bantuan22 = tkinter.Frame(my_frame_4_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_4_bantuan22.pack()
my_frame_4_bantuan22.pack_propagate(0)
my_frame_4_bantuan20 = tkinter.Frame(my_frame_4_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_4_bantuan20.pack()
my_frame_4_bantuan20.pack_propagate(0)
my_frame_4_bantuan23 = tkinter.Frame(my_frame_4_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_4_bantuan23.pack()
my_frame_4_bantuan23.pack_propagate(0)
my_frame_4_bantuan24 = tkinter.Frame(my_frame_4_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_4_bantuan24.pack()
my_frame_4_bantuan24.pack_propagate(0)

my_frame_4_text1 = tkinter.Label(my_frame_4_bantuan7, text='Kecepatan', background='#e2e6e2')
my_frame_4_text1.pack(side=tkinter.LEFT)
my_frame_4_text2 = tkinter.Label(my_frame_4_bantuan8, text='Sudut', background='#e2e6e2')
my_frame_4_text2.pack(side=tkinter.LEFT)
my_frame_4_text13 = tkinter.Label(my_frame_4_bantuan19, text='Voltase', background='#e2e6e2')
my_frame_4_text13.pack(side=tkinter.LEFT)
my_frame_4_text14 = tkinter.Label(my_frame_4_bantuan21, text='Sudut', background='#e2e6e2')
my_frame_4_text14.pack(side=tkinter.LEFT)
my_frame_4_text15 = tkinter.Label(my_frame_4_bantuan22, text='Voltase', background='#e2e6e2')
my_frame_4_text15.pack(side=tkinter.LEFT)
my_frame_4_text9 = tkinter.Label(my_frame_4_bantuan9, text=' = ', background='#e2e6e2')
my_frame_4_text9.pack(side=tkinter.LEFT)
my_frame_4_text10 = tkinter.Label(my_frame_4_bantuan10, text=' = ', background='#e2e6e2')
my_frame_4_text10.pack(side=tkinter.LEFT)
my_frame_4_text13 = tkinter.Label(my_frame_4_bantuan20, text=' = ', background='#e2e6e2')
my_frame_4_text13.pack(side=tkinter.LEFT)
my_frame_4_text16 = tkinter.Label(my_frame_4_bantuan23, text=' = ', background='#e2e6e2')
my_frame_4_text16.pack(side=tkinter.LEFT)
my_frame_4_text17 = tkinter.Label(my_frame_4_bantuan24, text=' = ', background='#e2e6e2')
my_frame_4_text17.pack(side=tkinter.LEFT)

my_frame_4_spinbox1 = tkinter.Entry(my_frame_4_bantuan9,width=15)
my_frame_4_spinbox1.configure(justify=tkinter.CENTER)
my_frame_4_spinbox1.pack()
my_frame_4_spinbox2 = tkinter.Entry(my_frame_4_bantuan10,width=15)
my_frame_4_spinbox2.configure(justify=tkinter.CENTER)
my_frame_4_spinbox2.pack()
my_frame_4_spinbox5 = tkinter.Entry(my_frame_4_bantuan20, width=15)
my_frame_4_spinbox5.configure(justify=tkinter.CENTER)
my_frame_4_spinbox5.pack()
my_frame_4_spinbox6 = tkinter.Entry(my_frame_4_bantuan23,width=15)
my_frame_4_spinbox6.configure(justify=tkinter.CENTER)
my_frame_4_spinbox6.pack()
my_frame_4_spinbox7 = tkinter.Entry(my_frame_4_bantuan24, width=15)
my_frame_4_spinbox7.configure(justify=tkinter.CENTER)
my_frame_4_spinbox7.pack()

#================================================BUTTON=========================

connect_btn = tkinter.Button(my_frame_1, text='Connect', width=9, state="disabled", command=connexion)
connect_btn.grid(column=3, row=2)
refresh_btn = tkinter.Button(my_frame_1, text='Refresh', width=9, command=update_coms)
refresh_btn.grid(column=3, row=3)
port_lable = Label(my_frame_1, text="Available Port(s): ", background='#e2e6e2')
port_lable.grid(column=1, row=2, pady=5, padx=5)
port_bd = Label(my_frame_1, text="Baude Rate: ", background='#e2e6e2')
port_bd.grid(column=1, row=3, pady=5, padx=5)
baud_select()
update_coms()

In [10]:
#========================================================mode test=======================================
my_frame_4 = tkinter.LabelFrame(part_1, width=150, height=100, text='''Test Method''', relief='groove', foreground="black",
                                background='#e2e6e2')
my_frame_4.pack(padx=5, side=tkinter.LEFT)
my_frame_4.pack_propagate(0)


#=============================================value modification motor 1==========================================
my_frame_2 = tkinter.LabelFrame(part_1, width=400, height=100, text='''Setting Value Motor 1''', relief='groove',
                                foreground="black", background='#e2e6e2')
my_frame_2.pack(side=tkinter.LEFT)
my_frame_2.pack_propagate(0)
my_frame_2_bantuan1 = tkinter.Frame(my_frame_2, width=85, height=50, background='#e2e6e2')
my_frame_2_bantuan1.pack(side=tkinter.LEFT)
my_frame_2_bantuan2 = tkinter.Frame(my_frame_2, width=65, height=50, background='#e2e6e2')
my_frame_2_bantuan2.pack(side=tkinter.LEFT)
my_frame_2_bantuan3 = tkinter.Frame(my_frame_2, width=50, height=50, background='#e2e6e2')
my_frame_2_bantuan3.pack(side=tkinter.LEFT)
my_frame_2_bantuan4 = tkinter.Frame(my_frame_2, width=50, height=50, background='#e2e6e2')
my_frame_2_bantuan4.pack(side=tkinter.LEFT)
my_frame_2_bantuan5 = tkinter.Frame(my_frame_2, width=65, height=50, background='#e2e6e2')
my_frame_2_bantuan5.pack(side=tkinter.LEFT)
my_frame_2_bantuan7 = tkinter.Frame(my_frame_2_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_2_bantuan7.pack()
my_frame_2_bantuan7.pack_propagate(0)
my_frame_2_bantuan8 = tkinter.Frame(my_frame_2_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_2_bantuan8.pack()
my_frame_2_bantuan8.pack_propagate(0)
my_frame_2_bantuan9 = tkinter.Frame(my_frame_2_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_2_bantuan9.pack()
my_frame_2_bantuan9.pack_propagate(0)
my_frame_2_bantuan10 = tkinter.Frame(my_frame_2_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_2_bantuan10.pack()
my_frame_2_bantuan10.pack_propagate(0)
my_frame_2_bantuan11 = tkinter.Frame(my_frame_2_bantuan3, width=50, height=25, background='#e2e6e2')
my_frame_2_bantuan11.pack()
my_frame_2_bantuan11.pack_propagate(0)
my_frame_2_bantuan12 = tkinter.Frame(my_frame_2_bantuan3, width=50, height=25, background='#e2e6e2')
my_frame_2_bantuan12.pack()
my_frame_2_bantuan12.pack_propagate(0)
my_frame_2_bantuan13 = tkinter.Frame(my_frame_2_bantuan4, width=35, height=25, background='#e2e6e2')
my_frame_2_bantuan13.pack()
my_frame_2_bantuan13.pack_propagate(0)
my_frame_2_bantuan14 = tkinter.Frame(my_frame_2_bantuan4, width=35, height=25, background='#e2e6e2')
my_frame_2_bantuan14.pack()
my_frame_2_bantuan14.pack_propagate(0)
my_frame_2_bantuan15 = tkinter.Frame(my_frame_2_bantuan5, width=65, height=25, background='#e2e6e2')
my_frame_2_bantuan15.pack()
my_frame_2_bantuan15.pack_propagate(0)
my_frame_2_bantuan16 = tkinter.Frame(my_frame_2_bantuan5, width=65, height=25, background='#e2e6e2')
my_frame_2_bantuan16.pack()
my_frame_2_bantuan16.pack_propagate(0)
my_frame_2_bantuan19 = tkinter.Frame(my_frame_2_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_2_bantuan19.pack()
my_frame_2_bantuan19.pack_propagate(0)
my_frame_2_bantuan20 = tkinter.Frame(my_frame_2_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_2_bantuan20.pack()
my_frame_2_bantuan20.pack_propagate(0)
my_frame_2_bantuan21 = tkinter.Frame(my_frame_2_bantuan3, width=50, height=25, background='#e2e6e2')
my_frame_2_bantuan21.pack()
my_frame_2_bantuan21.pack_propagate(0)

my_frame_2_text1 = tkinter.Label(my_frame_2_bantuan7, text='Kecepatan', background='#e2e6e2')
my_frame_2_text1.pack(side=tkinter.LEFT)
my_frame_2_text2 = tkinter.Label(my_frame_2_bantuan8, text='Sudut', background='#e2e6e2')
my_frame_2_text2.pack(side=tkinter.LEFT)
my_frame_2_text13 = tkinter.Label(my_frame_2_bantuan19, text='Anu', background='#e2e6e2')
my_frame_2_text13.pack(side=tkinter.LEFT)
my_frame_2_text3 = tkinter.Label(my_frame_2_bantuan11, text='Nodes', background='#e2e6e2')
my_frame_2_text3.pack(side=tkinter.LEFT)
my_frame_2_text4 = tkinter.Label(my_frame_2_bantuan12, text='Nodes', background='#e2e6e2')
my_frame_2_text4.pack(side=tkinter.LEFT)
my_frame_2_text14 = tkinter.Label(my_frame_2_bantuan21, text='Nodes', background='#e2e6e2')
my_frame_2_text14.pack(side=tkinter.LEFT)
my_frame_2_text5 = tkinter.Label(my_frame_2_bantuan13, text='P', background='#e2e6e2')
my_frame_2_text5.pack(side=tkinter.LEFT)
my_frame_2_text6 = tkinter.Label(my_frame_2_bantuan14, text='I', background='#e2e6e2')
my_frame_2_text6.pack(side=tkinter.LEFT)
my_frame_2_text9 = tkinter.Label(my_frame_2_bantuan9, text=' = ', background='#e2e6e2')
my_frame_2_text9.pack(side=tkinter.LEFT)
my_frame_2_text10 = tkinter.Label(my_frame_2_bantuan10, text=' = ', background='#e2e6e2')
my_frame_2_text10.pack(side=tkinter.LEFT)
my_frame_2_text11 = tkinter.Label(my_frame_2_bantuan15, text=' = ', background='#e2e6e2')
my_frame_2_text11.pack(side=tkinter.LEFT)
my_frame_2_text12 = tkinter.Label(my_frame_2_bantuan16, text=' = ', background='#e2e6e2')
my_frame_2_text12.pack(side=tkinter.LEFT)
my_frame_2_text13 = tkinter.Label(my_frame_2_bantuan20, text=' = ', background='#e2e6e2')
my_frame_2_text13.pack(side=tkinter.LEFT)
my_frame_2_spinbox1 = tkinter.Entry(my_frame_2_bantuan9, from_=None, to=None, width=4)
my_frame_2_spinbox1.configure(justify=tkinter.CENTER)
my_frame_2_spinbox1.pack(pady=1, side=tkinter.LEFT)
my_frame_2_spinbox2 = tkinter.Entry(my_frame_2_bantuan10, from_=None, to=None, width=4)
my_frame_2_spinbox2.configure(justify=tkinter.CENTER)
my_frame_2_spinbox2.pack(pady=1, side=tkinter.LEFT)
my_frame_2_spinbox5 = tkinter.Entry(my_frame_2_bantuan20, from_=None, to=None, width=4)
my_frame_2_spinbox5.configure(justify=tkinter.CENTER)
my_frame_2_spinbox5.pack(pady=1, side=tkinter.LEFT)

my_frame_2_spinbox3 = tkinter.Entry(my_frame_2_bantuan15, from_=None, to=None, width=4)
my_frame_2_spinbox3.configure(justify=tkinter.CENTER)
my_frame_2_spinbox3.pack(pady=1, side=tkinter.LEFT)
my_frame_2_spinbox4 = tkinter.Entry(my_frame_2_bantuan16, from_=None, to=None, width=4)
my_frame_2_spinbox4.configure(justify=tkinter.CENTER)
my_frame_2_spinbox4.pack(pady=1, side=tkinter.LEFT)

def kirim():
    data1 = ((my_frame_2_spinbox1.get()))
    data1a = format(data1, "b")
    data2 = ((my_frame_2_spinbox2.get()))
    data2a = format(data2, "b")
    data3 = ((my_frame_2_spinbox5.get()))
    data3a = format(data3, "b")
    data4 = ((my_frame_2_spinbox3.get()))
    data4a = format(data4, "b")
    data5 = ((my_frame_2_spinbox4.get()))
    data5a = format(data5, "b")

    k=0
    while k<5 :
        if k == 0:
            ser.write(data1a)
        elif k == 1:
            ser.write(data2a)
        elif k == 2:
            ser.write(data3a)
        elif k == 3:
            ser.write(data4a)  
        elif k == 4:
            ser.write(data5a)    
        time.sleep(1)
        k=k+1
    

my_frame_2_button = tkinter.Button(my_frame_2, text='SET1', width=9, command=kirim)
my_frame_2_button.pack()

#========================================================data value set motor 2==================================================
my_frame_3 = tkinter.LabelFrame(part_1, width=400, height=100, text='''Setting Value Motor 2''', relief='groove',
                                foreground="black", background='#e2e6e2')
my_frame_3.pack(side=tkinter.LEFT)
my_frame_3.pack_propagate(0)
my_frame_3_bantuan1 = tkinter.Frame(my_frame_3, width=85, height=50, background='#e2e6e2')
my_frame_3_bantuan1.pack(side=tkinter.LEFT)
my_frame_3_bantuan2 = tkinter.Frame(my_frame_3, width=65, height=50, background='#e2e6e2')
my_frame_3_bantuan2.pack(side=tkinter.LEFT)
my_frame_3_bantuan3 = tkinter.Frame(my_frame_3, width=50, height=50, background='#e2e6e2')
my_frame_3_bantuan3.pack(side=tkinter.LEFT)
my_frame_3_bantuan4 = tkinter.Frame(my_frame_3, width=50, height=50, background='#e2e6e2')
my_frame_3_bantuan4.pack(side=tkinter.LEFT)
my_frame_3_bantuan5 = tkinter.Frame(my_frame_3, width=65, height=50, background='#e2e6e2')
my_frame_3_bantuan5.pack(side=tkinter.LEFT)
my_frame_3_bantuan7 = tkinter.Frame(my_frame_3_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_3_bantuan7.pack()
my_frame_3_bantuan7.pack_propagate(0)
my_frame_3_bantuan8 = tkinter.Frame(my_frame_3_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_3_bantuan8.pack()
my_frame_3_bantuan8.pack_propagate(0)
my_frame_3_bantuan9 = tkinter.Frame(my_frame_3_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_3_bantuan9.pack()
my_frame_3_bantuan9.pack_propagate(0)
my_frame_3_bantuan10 = tkinter.Frame(my_frame_3_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_3_bantuan10.pack()
my_frame_3_bantuan10.pack_propagate(0)
my_frame_3_bantuan11 = tkinter.Frame(my_frame_3_bantuan3, width=50, height=25, background='#e2e6e2')
my_frame_3_bantuan11.pack()
my_frame_3_bantuan11.pack_propagate(0)
my_frame_3_bantuan12 = tkinter.Frame(my_frame_3_bantuan3, width=50, height=25, background='#e2e6e2')
my_frame_3_bantuan12.pack()
my_frame_3_bantuan12.pack_propagate(0)
my_frame_3_bantuan13 = tkinter.Frame(my_frame_3_bantuan4, width=35, height=25, background='#e2e6e2')
my_frame_3_bantuan13.pack()
my_frame_3_bantuan13.pack_propagate(0)
my_frame_3_bantuan14 = tkinter.Frame(my_frame_3_bantuan4, width=35, height=25, background='#e2e6e2')
my_frame_3_bantuan14.pack()
my_frame_3_bantuan14.pack_propagate(0)
my_frame_3_bantuan15 = tkinter.Frame(my_frame_3_bantuan5, width=65, height=25, background='#e2e6e2')
my_frame_3_bantuan15.pack()
my_frame_3_bantuan15.pack_propagate(0)
my_frame_3_bantuan16 = tkinter.Frame(my_frame_3_bantuan5, width=65, height=25, background='#e2e6e2')
my_frame_3_bantuan16.pack()
my_frame_3_bantuan16.pack_propagate(0)
my_frame_3_bantuan19 = tkinter.Frame(my_frame_3_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_3_bantuan19.pack()
my_frame_3_bantuan19.pack_propagate(0)
my_frame_3_bantuan20 = tkinter.Frame(my_frame_3_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_3_bantuan20.pack()
my_frame_3_bantuan20.pack_propagate(0)
my_frame_3_bantuan21 = tkinter.Frame(my_frame_3_bantuan3, width=50, height=25, background='#e2e6e2')
my_frame_3_bantuan21.pack()
my_frame_3_bantuan21.pack_propagate(0)

my_frame_3_text1 = tkinter.Label(my_frame_3_bantuan7, text='Kecepatan', background='#e2e6e2')
my_frame_3_text1.pack(side=tkinter.LEFT)
my_frame_3_text2 = tkinter.Label(my_frame_3_bantuan8, text='Sudut', background='#e2e6e2')
my_frame_3_text2.pack(side=tkinter.LEFT)
my_frame_3_text13 = tkinter.Label(my_frame_3_bantuan19, text='Anu', background='#e2e6e2')
my_frame_3_text13.pack(side=tkinter.LEFT)
my_frame_3_text3 = tkinter.Label(my_frame_3_bantuan11, text='Nodes', background='#e2e6e2')
my_frame_3_text3.pack(side=tkinter.LEFT)
my_frame_3_text4 = tkinter.Label(my_frame_3_bantuan12, text='Nodes', background='#e2e6e2')
my_frame_3_text4.pack(side=tkinter.LEFT)
my_frame_3_text14 = tkinter.Label(my_frame_3_bantuan21, text='Nodes', background='#e2e6e2')
my_frame_3_text14.pack(side=tkinter.LEFT)
my_frame_3_text5 = tkinter.Label(my_frame_3_bantuan13, text='P', background='#e2e6e2')
my_frame_3_text5.pack(side=tkinter.LEFT)
my_frame_3_text6 = tkinter.Label(my_frame_3_bantuan14, text='I', background='#e2e6e2')
my_frame_3_text6.pack(side=tkinter.LEFT)
my_frame_3_text9 = tkinter.Label(my_frame_3_bantuan9, text=' = ', background='#e2e6e2')
my_frame_3_text9.pack(side=tkinter.LEFT)
my_frame_3_text10 = tkinter.Label(my_frame_3_bantuan10, text=' = ', background='#e2e6e2')
my_frame_3_text10.pack(side=tkinter.LEFT)
my_frame_3_text11 = tkinter.Label(my_frame_3_bantuan15, text=' = ', background='#e2e6e2')
my_frame_3_text11.pack(side=tkinter.LEFT)
my_frame_3_text12 = tkinter.Label(my_frame_3_bantuan16, text=' = ', background='#e2e6e2')
my_frame_3_text12.pack(side=tkinter.LEFT)
my_frame_3_text13 = tkinter.Label(my_frame_3_bantuan20, text=' = ', background='#e2e6e2')
my_frame_3_text13.pack(side=tkinter.LEFT)
my_frame_3_spinbox1 = tkinter.Entry(my_frame_3_bantuan9, from_=None, to=None, width=4)
my_frame_3_spinbox1.configure(justify=tkinter.CENTER)
my_frame_3_spinbox1.pack(pady=1, side=tkinter.LEFT)
my_frame_3_spinbox2 = tkinter.Entry(my_frame_3_bantuan10, from_=None, to=None, width=4)
my_frame_3_spinbox2.configure(justify=tkinter.CENTER)
my_frame_3_spinbox2.pack(pady=1, side=tkinter.LEFT)
my_frame_3_spinbox5 = tkinter.Entry(my_frame_3_bantuan20, from_=None, to=None, width=4)
my_frame_3_spinbox5.configure(justify=tkinter.CENTER)
my_frame_3_spinbox5.pack(pady=1, side=tkinter.LEFT)

my_frame_3_spinbox3 = tkinter.Entry(my_frame_3_bantuan15, from_=None, to=None, width=4)
my_frame_3_spinbox3.configure(justify=tkinter.CENTER)
my_frame_3_spinbox3.pack(pady=1, side=tkinter.LEFT)
my_frame_3_spinbox4 = tkinter.Entry(my_frame_3_bantuan16, from_=None, to=None, width=4)
my_frame_3_spinbox4.configure(justify=tkinter.CENTER)
my_frame_3_spinbox4.pack(pady=1, side=tkinter.LEFT)

def predict():
    TD_Input = [] # Training Data Input
    TD_Target = [] # Training Data Target

    dataset = [[1],[kec,arus]] #fungsi y = x^2+10 dan random

    In_LF = np.zeros([1,2]) #inputnya 2, jml data 10

    for i in range(1):
        temp = dataset[i+1]
    for j in range(2):
        In_LF[i][j] = temp[j]
    
    data_masuk = np.size(In_LF[0])
    datanum = np.size(In_LF)/data_masuk
    TD_Input = In_LF

    Layer1 = 2
    Layer2 = 1
    Layer3 = 1
    Layer4 = 1

    W1_2 = [[-3.530076116406581],[-1.9350437958161328]]  # Weight Input Layer to Hidden Layer 1
    W2_3 = [[2.4756074199418405]] # Weight Hidden Layer 1 to Hidden Layer 2
    W3_4 = [[1.215934318147095]] 

    Th = np.zeros([Layer2+Layer3+Layer4])
    for i in range(int(np.size(Th))):
        Th[i] = 0.1
    
    Input = np.zeros([Layer1])
    OutHidden1 = np.zeros([Layer2])
    OutHidden2 = np.zeros([Layer3])
    OutLayer = np.zeros([Layer4])

    Input = np.zeros([Layer1])
    OutHidden1 = np.zeros([Layer2])
    OutHidden2 = np.zeros([Layer3])
    OutLayer = np.zeros([Layer4])

    for j in range(Layer1):
        Input[j] = TD_Input[0][j]

    # Layer 2
    for j in range(Layer2):
        temp = 0.0
    for k in range(Layer1):
        temp += Input[k]*W1_2[k][j]
    temp -= Th[j]
    OutHidden1[j] = 1/(1+np.exp(-temp))

    for j in range(Layer3):
        temp = 0.0
    for k in range(Layer2):
        temp += OutHidden1[k]*W2_3[k][j]
    temp -= Th[j]
    OutHidden2[j] = 1/(1+np.exp(-temp))

    for j in range(Layer4):
        temp = 0.0
    for k in range(Layer3):
        temp += OutHidden2[k] * W3_4[k][j]
    temp -= Th[j+Layer2+Layer3]
    OutLayer[j] = 1 / (1 + np.exp(-temp))

    my_frame_3_spinbox4.delete(0, tkinter.END)
    my_frame_3_spinbox4.insert(0, OutLayer)
    

def kirimi():
    predict()
    data1 = (my_frame_3_spinbox1.get())
    data2 = (my_frame_3_spinbox2.get())
    data3 = (my_frame_3_spinbox5.get())
    data4 = (my_frame_3_spinbox3.get())
    data5 = (my_frame_3_spinbox4.get())

    datakirim1 = data1+","+data2+","+data3+","+data4+","+data5

    ser.write(datakirim1)

my_frame_3_button = tkinter.Button(my_frame_3, text='SET2', width=9, command=kirimi)
my_frame_3_button.pack()

#==============================================read data motor 2============================================
my_frame_5 = tkinter.Frame(part_2, width=400, height=625, background='#e2e6e2')
my_frame_5 = tkinter.LabelFrame(part_2, width=200, height=200, text='''Data Reading : Motor 2''', relief='groove', foreground="black",
                                background='#e2e6e2')
my_frame_5.pack(pady=5, side=tkinter.LEFT)
my_frame_5.pack_propagate(0)

my_frame_5_bantuan1 = tkinter.Frame(my_frame_5, width=85, height=50, background='#e2e6e2')
my_frame_5_bantuan1.pack(side=tkinter.LEFT)
my_frame_5_bantuan2 = tkinter.Frame(my_frame_5, width=65, height=50, background='#e2e6e2')
my_frame_5_bantuan2.pack(side=tkinter.LEFT)
my_frame_5_bantuan9 = tkinter.Frame(my_frame_5_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_5_bantuan9.pack()
my_frame_5_bantuan9.pack_propagate(0)
my_frame_5_bantuan10 = tkinter.Frame(my_frame_5_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_5_bantuan10.pack()
my_frame_5_bantuan10.pack_propagate(0)

my_frame_5_bantuan7 = tkinter.Frame(my_frame_5_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_5_bantuan7.pack()
my_frame_5_bantuan7.pack_propagate(0)
my_frame_5_bantuan8 = tkinter.Frame(my_frame_5_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_5_bantuan8.pack()
my_frame_5_bantuan8.pack_propagate(0)
my_frame_5_bantuan19 = tkinter.Frame(my_frame_5_bantuan1, width=85, height=25, background='#e2e6e2')
my_frame_5_bantuan19.pack()
my_frame_5_bantuan19.pack_propagate(0)
my_frame_5_bantuan20 = tkinter.Frame(my_frame_5_bantuan2, width=65, height=25, background='#e2e6e2')
my_frame_5_bantuan20.pack()
my_frame_5_bantuan20.pack_propagate(0)

my_frame_5_text1 = tkinter.Label(my_frame_5_bantuan7, text='Kecepatan', background='#e2e6e2')
my_frame_5_text1.pack(side=tkinter.LEFT)
my_frame_5_text2 = tkinter.Label(my_frame_5_bantuan8, text='Sudut', background='#e2e6e2')
my_frame_5_text2.pack(side=tkinter.LEFT)
my_frame_5_text13 = tkinter.Label(my_frame_5_bantuan19, text='Voltase', background='#e2e6e2')
my_frame_5_text13.pack(side=tkinter.LEFT)
my_frame_5_text14 = tkinter.Label(my_frame_5_bantuan19, text='Voltase', background='#e2e6e2')
my_frame_5_text14.pack(side=tkinter.LEFT)
my_frame_5_text15 = tkinter.Label(my_frame_5_bantuan19, text='Voltase', background='#e2e6e2')
my_frame_5_text15.pack(side=tkinter.LEFT)
my_frame_5_text9 = tkinter.Label(my_frame_5_bantuan9, text=' = ', background='#e2e6e2')
my_frame_5_text9.pack(side=tkinter.LEFT)
my_frame_5_text10 = tkinter.Label(my_frame_5_bantuan10, text=' = ', background='#e2e6e2')
my_frame_5_text10.pack(side=tkinter.LEFT)
my_frame_5_text13 = tkinter.Label(my_frame_5_bantuan20, text=' = ', background='#e2e6e2')
my_frame_5_text13.pack(side=tkinter.LEFT)

my_frame_5_spinbox1 = tkinter.Entry(my_frame_5_bantuan9, from_=None, to=None, width=4)
my_frame_5_spinbox1.configure(justify=tkinter.CENTER)
my_frame_5_spinbox1.pack(pady=1, side=tkinter.LEFT)
my_frame_5_spinbox2 = tkinter.Entry(my_frame_5_bantuan10, from_=None, to=None, width=4)
my_frame_5_spinbox2.configure(justify=tkinter.CENTER)
my_frame_5_spinbox2.pack(pady=1, side=tkinter.LEFT)
my_frame_5_spinbox5 = tkinter.Entry(my_frame_5_bantuan20, from_=None, to=None, width=4)
my_frame_5_spinbox5.configure(justify=tkinter.CENTER)
my_frame_5_spinbox5.pack(pady=1, side=tkinter.LEFT)



windows.mainloop()

13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7
13,7,8,7,7